**This notebook is a comparison between 3 feature subsets on CV=5 for all horizons.**

Feature subset 1: Original series with lags 1 and 2 as features.

Feature subset 2: Exogenous + TsFresh Feature Generation + Filter.

Feature subset 3 is Exogenous + TsFresh Feature Generation + Filter + ARFS Leshy (Boruta) Selection.

--
Important: Filter means eliminating features generated by TSFresh based on collinearity, cardinality, missing values and unique values.

In [ ]:
!git clone https://github.com/Cajeux1999/AEMO-Solar-Energy-Forecasting.git -q
!pip install "u8darts[all]"

import pandas as pd
import numpy as np

from sklearn.linear_model import Ridge
import time
import matplotlib.pyplot as plt

from darts import concatenate, TimeSeries
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from darts.dataprocessing.transformers import Scaler
from darts.dataprocessing.transformers import MissingValuesFiller
from darts.metrics import rmse, mae, mape, smape, r2_score
from darts.models import LightGBMModel, BlockRNNModel, XGBModel, NHiTSModel, RNNModel, RandomForestModel, SKLearnModel, NBEATSModel

In [2]:
REPO_NAME = "AEMO-Solar-Energy-Forecasting"
%cd {REPO_NAME}

from src import config, utils
train_size = config.TRAIN_SIZE
horizons = config.HORIZONS
strides = config.STRIDES


#Defining the filepath to load the dataset
filepath_processed = '/content/AEMO-Solar-Energy-Forecasting/data/processed/'
filepath_raw = '/content/AEMO-Solar-Energy-Forecasting/data/raw/'

/content/AEMO-Solar-Energy-Forecasting


## Subset 1: Original series with lags 1 and 2 as features.

In [6]:
y = pd.read_csv(filepath_raw+'nsw_solar.csv').drop(columns='Unnamed: 0')

scaler_y = Scaler()
scaler_dict_s1 = {'series': scaler_y}

y_train = TimeSeries.from_dataframe(
    y[:train_size],
    value_cols=['y'],
    time_col='ds',
    freq='30min'
)

In [7]:
def objective_rr_s1(h=None, stride=None):

    #Model construction

    ridge_model = SKLearnModel(

        model=Ridge(),

        lags=[-1,-2],
        output_chunk_length=h,
    )


    n_time_series = 0
    window_rmse = 0
    rmse_windows = 0


    # CV validation, based on 5 windows.

    start = time.time()

    backtest = ridge_model.historical_forecasts(y_train,
                                                forecast_horizon=h,
                                                train_length=2003,
                                                last_points_only=False,
                                                stride=stride,
                                                retrain=True,
                                                data_transformers=scaler_dict_s1,
                                                show_warnings=False)

    training_time = time.time() - start

    # RMSE Calculation and return.
    for ts in backtest:
      st = ts.start_time()
      en = ts.end_time()
      ts_train = y_train.slice(st, en)

      if len(ts_train) == len(ts):

          window_rmse = rmse(ts_train, ts)
          rmse_windows += window_rmse
          n_time_series +=1

      else:
          print(f"Skipping window {st} to {en} due to length mismatch")

    average_rmse = rmse_windows / n_time_series

    return np.array([average_rmse, training_time])

In [24]:
all_horizons = []

for h,s in zip(horizons,strides):

  s1=objective_rr_s1(h=h,stride=s)
  all_horizons.append(s1)

media_rmse = np.mean(all_horizons, axis=0)[0]
media_tempo = np.mean(all_horizons, axis=0)[1]

print(f"RMSE Average across all horizons: {media_rmse:.4f}")
print(f"Average training time: {media_tempo:.4f} sec")

RMSE Average across all horizons: 615.5028
Average training time: 0.2099 sec


## Subset 2: Exogenous + TsFresh Feature Generation + Filter.

In [33]:
y = pd.read_csv(filepath_raw+'nsw_solar.csv').drop(columns='Unnamed: 0')
X_s2 = pd.read_csv(filepath_processed+'df_features_preprocessed.csv').drop(columns='Unnamed: 0')

scaler_covariates_s2 = Scaler(global_fit=False)
scaler_y = Scaler()

scaler_dict_s2 = {'series': scaler_y, 'past_covariates': scaler_covariates_s2}

In [34]:
past_covariates_list = [col for col in X_s2 if col not in['ds','halfhour_sin','halfhour_cos','day_sin','day_cos']]

train_past_covariates_s2 = TimeSeries.from_dataframe(
    X_s2[:train_size],
    value_cols=past_covariates_list,
    time_col='ds', # Use 'ds' como a coluna de tempo (índice)
    freq='30min'
)

future_covariates_s2 = TimeSeries.from_dataframe(
    X_s2,
    value_cols=['halfhour_sin','halfhour_cos','day_sin','day_cos'],
    time_col='ds', # Use 'ds' como a coluna de tempo (índice)
    freq='30min'
)

y_train = TimeSeries.from_dataframe(
    y[:train_size],
    value_cols=['y'],
    time_col='ds',
    freq='30min'
)

In [35]:
transformer_s2 = MissingValuesFiller()
train_past_covariates_s2_transf = transformer_s2.transform(train_past_covariates_s2)

In [38]:
def objective_rr_s2(h=None, stride=None):


    #Model construction

    ridge_model = SKLearnModel(

        model=Ridge(),

        lags_past_covariates=2,
        lags_future_covariates=[0],

        output_chunk_length=h,
    )


    n_time_series = 0
    window_rmse = 0
    rmse_windows = 0


    # CV validation, based on 5 windows.

    start = time.time()

    backtest = ridge_model.historical_forecasts(y_train,
                                                past_covariates=train_past_covariates_s2_transf,
                                                future_covariates=future_covariates_s2,
                                                forecast_horizon=h,
                                                train_length=2003,
                                                last_points_only=False,
                                                stride=stride,
                                                retrain=True,
                                                data_transformers=scaler_dict_s2,
                                                show_warnings=False)

    training_time = time.time() - start

    # RMSE Calculation and return.
    for ts in backtest:
      st = ts.start_time()
      en = ts.end_time()
      ts_train = y_train.slice(st, en)

      if len(ts_train) == len(ts):

          window_rmse = rmse(ts_train, ts)
          rmse_windows += window_rmse
          n_time_series +=1

      else:
          print(f"Skipping window {st} to {en} due to length mismatch")

    average_rmse = rmse_windows / n_time_series

    return np.array([average_rmse, training_time])

In [39]:
all_horizons = []

for h,s in zip(horizons,strides):

  s2=objective_rr_s2(h=h,stride=s)
  all_horizons.append(s2)

media_rmse = np.mean(all_horizons, axis=0)[0]
media_tempo = np.mean(all_horizons, axis=0)[1]

print(f"RMSE Average across all horizons: {media_rmse:.4f}")
print(f"Average training time: {media_tempo:.4f} sec")

RMSE Average across all horizons: 895.1690
Average training time: 2.1728 sec


## Subset 3: Exogenous + TsFresh Feature Generation + Filter + ARFS Leshy (Boruta) Selection.

In [47]:
y = pd.read_csv(filepath_raw+'nsw_solar.csv').drop(columns='Unnamed: 0')
X_s3 = pd.read_csv(filepath_processed+'X.csv').drop(columns='Unnamed: 0')

scaler_covariates_s3 = Scaler(global_fit=False)
scaler_y = Scaler()

scaler_dict_s3 = {'series': scaler_y, 'past_covariates': scaler_covariates_s3}

In [48]:
past_covariates_list_s3 = [col for col in X_s3 if col not in['ds','halfhour_sin','halfhour_cos']]

train_past_covariates_s3 = TimeSeries.from_dataframe(
    X_s3[:train_size],
    value_cols=past_covariates_list_s3,
    time_col='ds', # Use 'ds' como a coluna de tempo (índice)
    freq='30min'
)

future_covariates_s3 = TimeSeries.from_dataframe(
    X_s3,
    value_cols=['halfhour_sin','halfhour_cos'],
    time_col='ds', # Use 'ds' como a coluna de tempo (índice)
    freq='30min'
)

y_train = TimeSeries.from_dataframe(
    y[:train_size],
    value_cols=['y'],
    time_col='ds',
    freq='30min'
)

In [49]:
transformer_s3 = MissingValuesFiller()
train_past_covariates_s3_transf = transformer_s3.transform(train_past_covariates_s3)

In [50]:
def objective_rr_s3(h=None, stride=None):


    #Model construction

    ridge_model = SKLearnModel(

        model=Ridge(),

        lags_past_covariates=2,
        lags_future_covariates=[0],

        output_chunk_length=h,
    )


    n_time_series = 0
    window_rmse = 0
    rmse_windows = 0


    # CV validation, based on 5 windows.

    start = time.time()

    backtest = ridge_model.historical_forecasts(y_train,
                                                past_covariates=train_past_covariates_s3_transf,
                                                future_covariates=future_covariates_s3,
                                                forecast_horizon=h,
                                                train_length=2003,
                                                last_points_only=False,
                                                stride=stride,
                                                retrain=True,
                                                data_transformers=scaler_dict_s3,
                                                show_warnings=False)

    training_time = time.time() - start

    # RMSE Calculation and return.
    for ts in backtest:
      st = ts.start_time()
      en = ts.end_time()
      ts_train = y_train.slice(st, en)

      if len(ts_train) == len(ts):

          window_rmse = rmse(ts_train, ts)
          rmse_windows += window_rmse
          n_time_series +=1

      else:
          print(f"Skipping window {st} to {en} due to length mismatch")

    average_rmse = rmse_windows / n_time_series

    return np.array([average_rmse, training_time])

In [51]:
all_horizons = []

for h,s in zip(horizons,strides):

  s3=objective_rr_s3(h=h,stride=s)
  all_horizons.append(s3)

media_rmse = np.mean(all_horizons, axis=0)[0]
media_tempo = np.mean(all_horizons, axis=0)[1]

print(f"RMSE Average across all horizons: {media_rmse:.4f}")
print(f"Average training time: {media_tempo:.4f} sec")

RMSE Average across all horizons: 501.2987
Average training time: 0.4044 sec
